In [14]:
import json
import numpy as np
from scipy.special import expit  # Sigmoid function
from torchvision import datasets, transforms

# Load model from JSON file
def load_model(json_path):
    with open(json_path, 'r') as f:
        model = json.load(f)
    return model

# Define activation functions
activation_functions = {
    "relu": lambda x: np.maximum(0, x),
    "sigmoid": expit,
    "tanh": np.tanh,
    "linear": lambda x: x,
    "softmax": lambda x: np.exp(x) / np.sum(np.exp(x), axis=0, keepdims=True)  # Softmax function fixed for 1D input
}

# Define standalone functions for each neuron
class MLP_Network:
    def __init__(self, model_json):
        self.layers = []
        self.activations = []
        
        for layer_key in sorted(model_json.keys(), key=lambda x: int(x.split('_')[1])):
            layer = model_json[layer_key]
            self.layers.append((np.array(layer["weights"]).T, np.array(layer["biases"])))  # Transpose weights
            self.activations.append(activation_functions[layer["activation"]])
    
    def forward(self, x):
        for (weights, biases), activation in zip(self.layers, self.activations):
            x = activation(np.dot(x, weights) + biases)
        return x

# Load the model from the provided JSON file
model_path = "model.json"
model_json = load_model(model_path)
mlp = MLP_Network(model_json)

# Load MNIST data using PyTorch datasets
mnist_test = datasets.MNIST(root="./data", train=False, transform=transforms.ToTensor(), download=True)

# Test model on a single MNIST image
image, label = mnist_test[0]
image = image.numpy().reshape(-1)  # Flatten the 28x28 image into a vector
prediction = mlp.forward(image)
predicted_label = np.argmax(prediction)

print(f"Predicted label: {predicted_label}, Actual label: {label}")

Predicted label: 7, Actual label: 7


In [25]:
import json
import numpy as np
from scipy.special import expit  # Sigmoid function
from torchvision import datasets, transforms

# Load model from JSON file
def load_model(json_path):
    with open(json_path, 'r') as f:
        model = json.load(f)
    return model

# Define activation functions
activation_functions = {
    "relu": lambda x: np.maximum(0, x),
    "sigmoid": expit,
    "tanh": np.tanh,
    "linear": lambda x: x,
    "softmax": lambda x: np.exp(x) / np.sum(np.exp(x), axis=0, keepdims=True)  # Softmax function fixed for 1D input
}

# Define standalone functions for each neuron
class MLP_Network:
    def __init__(self, model_json):
        self.layers = []
        self.activations = []
        self.outputs = {}  # Dictionary to store layer outputs
        
        for layer_key in sorted(model_json.keys(), key=lambda x: int(x.split('_')[1])):
            layer = model_json[layer_key]
            self.layers.append((np.array(layer["weights"]).T, np.array(layer["biases"])))  # Transpose weights
            self.activations.append(activation_functions[layer["activation"]])
    
    def forward(self, x):
        self.outputs["input"] = x  # Store input layer
        for i, ((weights, biases), activation) in enumerate(zip(self.layers, self.activations)):
            x = activation(np.dot(x, weights) + biases)
            self.outputs[f"layer_{i}"] = x  # Store each layer's output
        return x

# Load the model from the provided JSON file
model_path = "model.json"
model_json = load_model(model_path)
mlp = MLP_Network(model_json)

# Load MNIST data using PyTorch datasets
mnist_test = datasets.MNIST(root="./data", train=False, transform=transforms.ToTensor(), download=True)

# Test model on a single MNIST image
image, label = mnist_test[0]
image = image.numpy().reshape(-1)  # Flatten the 28x28 image into a vector
prediction = mlp.forward(image)
predicted_label = np.argmax(prediction)

# Print stored outputs from each layer
for layer, output in mlp.outputs.items():
    print(f"{layer} output: {output[:5]}")  # Print first 5 values of each layer output for brevity

print(f"Predicted label: {predicted_label}, Actual label: {label}")

input output: [0. 0. 0. 0. 0.]
layer_0 output: [0.         1.26142723 0.         0.         3.11774806]
layer_1 output: [1.60325687e-04 5.34613357e-05 2.66673920e-02 5.01315493e-02
 9.32749863e-06]
Predicted label: 7, Actual label: 7


In [29]:
print(len(model_json['layer_0']['weights'][1]))

784
